## Configuration and deployment of multiple 5G UEs of Open Air Interface (OAI)
## Objectives
- Configure and deploy multiple UEs 

## Multiple UE Configuration

To connect multiple UEs, you'll need to set up a network namespace in Linux, as well as add route configurations, by following the commands below:

**Step 1:** Creating a namespace:

sudo ip netns add oai-ue2

**Step 2:** Creating a pair of virtual interfaces (veth):

sudo ip link add veth0 type veth peer name veth1

**Step 3:** Moving one interface to the created namespace:

sudo ip link set veth1 netns oai-ue2

**Step 4:** Configuring an IP address for veth0 interface:

sudo ip addr add 192.168.1.1/24 dev veth0

**Step 5:** Configuring an IP address for veth1 interface within the namespace:

sudo ip netns exec oai-ue2 ip addr add 192.168.1.2/24 dev veth1

**Step 6:** Bringing up the veth0 interface:

sudo ip link set veth0 up

**Step 7:** Bringing up the veth1 interface within the namespace:

sudo ip netns exec oai-ue2 ip link set veth1 up

**Step 8:** Adding a route within the namespace:

sudo ip netns exec oai-ue2 ip route add 192.168.70.128/26 via 192.168.1.1 dev veth1

**Step 9:** Enabling packet forwarding on the system:

sudo sysctl net.ipv4.conf.all.forwarding=1

**Step 10:** Setting default policy for packet forwarding to ACCEPT:

sudo iptables -P FORWARD ACCEPT

**Step 11:** Enabling packet forwarding within the namespace:

sudo ip netns exec oai-ue2 sysctl net.ipv4.conf.all.forwarding=1

**Step 12:** Setting default policy for packet forwarding within the namespace to ACCEPT::

sudo ip netns exec oai-ue2 iptables -P FORWARD ACCEPT


## Multiple UE Deployment

**Step 1:** The deployment of multiple UEs is done in a similar way to a single UE, but it is necessary to enter the namespace before deploying. Run the following command:

sudo ip netns exec oai-ue2 bash

**Attention:** Remember to change the namespace according to the one you created earlier, in the example above "oai-ue2"

**Step 2:** To deploy the UE, type in terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --uicc0.imsi 001010000000002

**Attention:** Note that in the deployment command, the parameter "--uicc0.imsi 001010000000002" has been added, referring to the IMSI of the second UE, you MUST enter an IMSI that is present in the database (as shown in the "UE Configuration" section) and that IS NOT BEING USED BY ANOTHER UE.